In [ ]:
import re
import datetime
import ast
from collections import Counter
from pprint import pprint

import scipy.stats
import business_models
import tqdm.notebook
import pandas as pd
import py3dbp
import numpy as np
import matplotlib.pyplot as plt
from nile.api.v1 import (
    filters as nf,
    aggregators as na,
    extractors as ne,
    Record
)
from qb2.api.v1 import (filters as qf, typing)

from projects.common.nile import dates
from projects.common.nile import test_utils
from projects.product.c2c_tariffs import project_config

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

In [ ]:
cluster = project_config.get_project_cluster(
    parallel_operations_limit=10,
)

## Calculate AB sample size

In [ ]:
table_k2_orders_path = (
    '//home/taxi-delivery/analytics/dev/product/product_adhoc_exps/'
    'k2_orders_w_taxi_2021-06-03_2021-06-21'
)

df_k2_orders = test_utils.decode_dataframe(
    cluster
    .read(table_k2_orders_path)
    .as_dataframe()
)

In [ ]:
df_k2_orders.head()

In [ ]:
def get_first_nonzero(x):
    nonzero_list = list(filter(None, set(x.values)))
    if nonzero_list:
        return nonzero_list[0]
    return None

In [ ]:
df_k2 = df_k2_orders.groupby(['market_order_id']).agg({
    'lavka_app_platform': get_first_nonzero,
    'lavka_flag': lambda x: x.dropna().values.max() if x.dropna().values.any() else None,
    'lavka_local_created_dttm': lambda x: x.dropna().values.min() if x.dropna().values.any() else None,
    'market_app_platform': get_first_nonzero,
    'market_order_cancelled_flag': lambda x: x.dropna().values.max() if x.dropna().values.any() else None,
    'market_order_delivered_datetime': lambda x: x.dropna().values.max() if x.dropna().values.any() else None,
    'market_order_delivered_flag': lambda x: x.dropna().values.min() if x.dropna().values.any() else None,
    'market_to_pp_dttm': lambda x: x.dropna().values.max() if x.dropna().values.any() else None,
    'max_app_version': lambda x: x.dropna().values.max() if x.dropna().values.any() else None,
    'min_app_version': lambda x: x.dropna().values.min() if x.dropna().values.any() else None,
    'trip_cnt': get_first_nonzero,
    'user_phone_pd_id': get_first_nonzero,
    'yandex_uid': get_first_nonzero,
}).reset_index()

In [ ]:
df_k2.head()

In [ ]:
df_k2['market_to_pp_flag'] = (df_k2['market_to_pp_dttm']!='1970-01-01T00:00:00').values
df_k2['cancelled_after_pp_flag'] = (
    df_k2['market_to_pp_flag'] & df_k2['market_order_cancelled_flag']
).values

In [ ]:
df_k2['pp_to_delivered_sec'] = (
    df_k2[df_k2['market_order_delivered_datetime'].astype(bool)]['market_order_delivered_datetime'].apply(
        lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')
    )
    - df_k2[df_k2['market_order_delivered_datetime'].astype(bool)]['market_to_pp_dttm'].apply(
        lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')
    )
).apply(lambda x: x.seconds)

In [ ]:
df_k2.head()

In [ ]:
df_k2.shape

In [ ]:
df_k2_users = df_k2[
    ~pd.isnull(df_k2['user_phone_pd_id'])
].groupby('user_phone_pd_id').agg({
    'market_order_id': 'count',
    'lavka_flag': lambda x: x.astype(bool).astype(int).sum(),
    'market_order_cancelled_flag': lambda x: x.astype(bool).astype(int).sum(),
    'market_order_delivered_flag': lambda x: x.astype(bool).astype(int).sum(),
    'market_to_pp_flag': lambda x: x.astype(bool).astype(int).sum(),
    'cancelled_after_pp_flag': lambda x: x.astype(bool).astype(int).sum(),
    'pp_to_delivered_sec': 'mean',
    'trip_cnt': lambda x: list(filter(None, set(x.values)))[0] if list(filter(None, set(x.values))) else None,
}).reset_index()

In [ ]:
# df_k2_users = df_k2_users[
#     ((df_k2_users['max_app_version'] >= '600.40.0')
#     & (df_k2_users['lavka_app_platform']=='superapp_ios'))
#     | ((df_k2_users['max_app_version'] >= '4.38.0')
#        & (df_k2_users['lavka_app_platform']=='superapp_android'))
#     | (pd.isnull(df_k2_users['max_app_version']))
# ]

In [ ]:
df_k2_users['trip_cnt'].fillna(0, inplace=True)

In [ ]:
df_k2_users.head()

In [ ]:
df_k2_users['market_order_id'].sum()

In [ ]:
df_k2_users['lavka_flag'].sum()

In [ ]:
df_k2_users['market_order_cancelled_flag'].sum()

In [ ]:
df_k2_users['market_order_delivered_flag'].sum()

In [ ]:
df_k2_users['market_to_pp_flag'].sum()

In [ ]:
df_k2_users['cancelled_after_pp_flag'].sum()

### Split to test and control

In [ ]:
import hashlib
import sys

def exp3_hash(arg, salt):
    sha1 = hashlib.sha1()
    sha1.update(salt)
    if isinstance(arg, bytes):
        sha1.update(arg)
    elif isinstance(arg, int):
        sha1.update(arg.to_bytes(8, byteorder=sys.byteorder))
    else:
        raise ValueError('Unknown type')
    res = sha1.hexdigest()[:16]
    return int(res, 16)

In [ ]:
df_k2_users['hash_value'] = df_k2_users['user_phone_pd_id'].apply(
    lambda x: exp3_hash(test_utils.to_bytes(x), salt=b'on_demand_shipments_v2_2021-06')%100
)

In [ ]:
df_k2_users['group_id'] = df_k2_users['hash_value'].apply(lambda x: 1 if x <50 else 0)

In [ ]:
df_k2_users['market_order_delivered_flag'].sum() / df_k2_users['market_to_pp_flag'].sum()

In [ ]:
df_k2_users['market_order_delivered_flag'].sum() / df_k2_users['market_order_id'].sum()

In [ ]:
for gid in [0, 1]:
    print(df_k2_users[df_k2_users['group_id']==gid]['market_order_delivered_flag'].sum()
          / df_k2_users[df_k2_users['group_id']==gid]['market_order_id'].sum())

for gid in [0, 1]:    
    print(df_k2_users[df_k2_users['group_id']==gid]['market_order_delivered_flag'].sum()
          / df_k2_users[df_k2_users['group_id']==gid]['market_to_pp_flag'].sum())

for gid in [0, 1]:    
    print(df_k2_users[df_k2_users['group_id']==gid]['cancelled_after_pp_flag'].sum()
          / df_k2_users[df_k2_users['group_id']==gid]['market_to_pp_flag'].sum())
    
for gid in [0, 1]:    
    print(df_k2_users[df_k2_users['group_id']==gid]['pp_to_delivered_sec'].mean())
    
    

In [ ]:
print(exp3_hash(b'6ac2fcff96974a88a9930da7c6ec6993', b'on_demand_shipments_v2_2021-06')%100 < 50)
print(exp3_hash(b'6c6f02d18ab34a76b3b238ef62f963bd', b'on_demand_shipments_v2_2021-06')%100 < 50)
print(exp3_hash(b'79e9d68ff882486ca72f2264657bb368', b'on_demand_shipments_v2_2021-06')%100 < 50)
print(exp3_hash(b'6c6f02d18ab34a76b3b238ef62f963bd', b'on_demand_shipments_v2_2021-06')%100 < 50)
print(exp3_hash(b'3f1889263af045e19f90a5b34d38470a', b'on_demand_shipments_v2_2021-06')%100 < 50)
print(exp3_hash(b'e836060a0a3e4883b94569899478189b', b'on_demand_shipments_v2_2021-06')%100 < 50)
print(exp3_hash(b'02ba768d02a14ca0ad10895b69cdeb0a', b'on_demand_shipments_v2_2021-06')%100 < 50)
print(exp3_hash(b'd3d0380db95442c39631eb8a3e879c25', b'on_demand_shipments_v2_2021-06')%100 < 50)
print(exp3_hash(b'b2ed8992a12245688664860378cf423f', b'on_demand_shipments_v2_2021-06')%100 < 50)

### Bootstrap AB

In [ ]:
EXP_GROUP_ID = 1
CON_GROUP_ID = 0

METRIC_AGG_DICT = {
    'market_order_id': 'sum',
    'market_order_cancelled_flag': 'sum',
    'market_order_delivered_flag': 'sum',
    'market_to_pp_dttm': 'sum',
}

In [ ]:
def split_users(
    df, exp_group_id, con_group_id
):
    
    treatment_group = (
        df[df['group_id']==exp_group_id]
    )
    control_group = (
        df[df['group_id']==con_group_id]
    )
    
    return treatment_group, control_group

def get_bootstrap_data(
    treatment_group, control_group, n_iter=1000
):
    
    bs_list = []
    for i in tqdm.notebook.tqdm(range(n_iter)):
        iteration_dict = {'iter_num': i}
        tg_bs = treatment_group.sample(frac=1, replace=True, random_state=i)
        cg_bs = control_group.sample(frac=1, replace=True, random_state=i)
        
        iteration_dict['order_to_delivered_diff_abs'] = round(
            tg_bs['market_order_delivered_flag'].sum()
            / tg_bs['market_order_id'].sum()
            - cg_bs['market_order_delivered_flag'].sum()
            / cg_bs['market_order_id'].sum()
        , 4)
        iteration_dict['pp_to_delivered_diff_abs'] = round(
            tg_bs['market_order_delivered_flag'].sum()
            / tg_bs['market_to_pp_flag'].sum()
            - cg_bs['market_order_delivered_flag'].sum()
            / cg_bs['market_to_pp_flag'].sum()
        , 4)
        iteration_dict['pp_to_cancelled_diff_abs'] = round(
            tg_bs['cancelled_after_pp_flag'].sum()
            / tg_bs['market_to_pp_flag'].sum()
            - cg_bs['cancelled_after_pp_flag'].sum()
            / cg_bs['market_to_pp_flag'].sum()
        , 4)
        iteration_dict['pp_to_delivered_sec_diff_abs'] = round(
            tg_bs['pp_to_delivered_sec'].mean()
            - cg_bs['pp_to_delivered_sec'].mean()
        , 4)
        
        
        iteration_dict['trip_cnt_diff_rel'] = round(
            (tg_bs['trip_cnt'].sum() - cg_bs['trip_cnt'].sum())
            / cg_bs['trip_cnt'].sum()
        , 4)
        bs_list.append(iteration_dict)

    return pd.DataFrame(bs_list)
        
    
def get_conf_intervals(bootstrap_data, metric_list, width=0.9):
    conf_int_dict = {}
    for metric in metric_list:
        conf_int_dict['{}_left'.format(metric)] = np.percentile(
            bootstrap_data[metric],
            ((1-width)/2)*100
        )
        conf_int_dict['{}_right'.format(metric)] = np.percentile(
            bootstrap_data[metric],
            (width + (1-width)/2)*100
        )
        conf_int_dict['{}_mean'.format(metric)] = np.mean(
            bootstrap_data[metric],
        )
        conf_int_dict['{}_median'.format(metric)] = np.median(
            bootstrap_data[metric],
        )
        
    return conf_int_dict


def get_test_results(
    userdays,
    dt_start, dt_end,
    metric_agg_dict, metric_list,
    exp_group_id, con_group_id,
):
    treatment_group, control_group = split_users(
        userdays=userdays,
        dt_start=dt_start,
        dt_end=dt_end,
        metric_agg_dict=metric_agg_dict,
        exp_group_id=exp_group_id,
        con_group_id=con_group_id,
    )

    bs_data = get_bootstrap_data(
        treatment_group=treatment_group,
        control_group=control_group,
        metric_agg_dict=metric_agg_dict
    )

    conf_int = get_conf_intervals(
        bootstrap_data=bs_data,
        metric_list=metric_list,
    )

    return conf_int

In [ ]:
treatment_group, control_group = split_users(
    df_k2_users, exp_group_id=EXP_GROUP_ID, con_group_id=CON_GROUP_ID
)

In [ ]:
treatment_group.shape, control_group.shape

In [ ]:
bs_data = get_bootstrap_data(treatment_group, control_group)

In [ ]:
conf_ints = get_conf_intervals(
    bs_data,
    ['order_to_delivered_diff_abs',
     'pp_to_delivered_diff_abs',
     'pp_to_cancelled_diff_abs',
     'trip_cnt_diff_rel',
     'pp_to_delivered_sec_diff_abs']
)

In [ ]:
conf_ints

In [ ]:
order_to_delivered = (
    control_group['market_order_delivered_flag'].sum() / control_group['market_order_id'].sum()
)
pp_to_delivered = (
    control_group['market_order_delivered_flag'].sum() / control_group['market_to_pp_flag'].sum()
)
pp_to_cancelled = (
    control_group['cancelled_after_pp_flag'].sum() / control_group['market_to_pp_flag'].sum()
)
pp_to_delivered_sec = control_group['pp_to_delivered_sec'].mean()


trip_diff = (
    (treatment_group['trip_cnt'].sum() - control_group['trip_cnt'].sum())
    / control_group['trip_cnt'].sum()
)

In [ ]:
pd.DataFrame(
    {'diff': conf_ints.values(),
     'abs': [order_to_delivered]*4
     + [pp_to_delivered]*4
     + [pp_to_cancelled]*4
     + [trip_diff]*4
     + [pp_to_delivered_sec]*4},
    index=conf_ints.keys()
).round(4)

### MannWhitney

In [ ]:
from scipy.stats import mannwhitneyu, ttest_ind

In [ ]:
print(mannwhitneyu(treatment_group['trip_cnt'], control_group['trip_cnt'], alternative='less'))